In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

from collections import namedtuple

In [41]:
def get_data(preprocess = True):
    if preprocess :
        data = pd.read_csv('./datas/naver_news/preprocessing_tok_naver_data.csv')
    else:
        economy = pd.read_csv('./datas/naver_news/economy.csv', header=None)
        policy = pd.read_csv('./datas/naver_news/policy.csv', header=None)
        it = pd.read_csv('./datas/naver_news/it.csv', header=None)

        columns = ['date', 'category', 'company', 'title', 'content', 'url']
        economy.columns = columns
        policy.columns = columns
        it.columns = columns

        data = pd.concat([economy, policy, it], axis = 0)
        data.reset_index(drop=True, inplace=True)
    
    return data

In [42]:
def get_preprocessing_data(data):
    data.drop(['date', 'company', 'url'], axis = 1, inplace =True)
    
    category_mapping = {
    '경제' : 0,
    '정치' : 1,
    'IT과학' : 2
    }

    data['category'] = data['category'].map(category_mapping)
    data['title_content'] = data['title'] + " " + data['content']
    data.drop(['title', 'content'], axis = 1, inplace = True)
    return data

In [79]:
def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    return data_doc

In [44]:
def training_doc2vec(data):
    model = Doc2Vec(data, vector_size=128, window=7, epochs=80, min_count=0, workers=4)
    return model

In [91]:
def get_recommened_contents(user, data_doc, model):
    scores = []

    for tags, text in data_doc:
        trained_doc_vec = model.docvecs[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:10]
    
    return data.iloc[scores, :]

In [59]:
def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

In [45]:
data = get_data()

In [46]:
data.head()

,category,title_content,mecab_tok
0,0,포스코ICT 4분기부터 실적 본격 개선 비용효율화 본격화 그룹 계열사와의 시너지 발...,포스코 ICT 분기 실 본격 개선 비용 효율 본격화 그룹 계열사 시너지 발생 헤럴드...
1,0,위메프 리퍼데이 바자회 수익금 소외계층 지원 위메프가 리퍼데이 바자회 ‘아름다운가게...,위메프 리퍼 데이 바자회 수익금 소외 계층 지원 위메프 리퍼 데이 바자회 가게 위메...
2,0,호반건설 광주시 ‘계림1구역 정비사업’ 수주…공사비 2700억원 최고 33층 총 9...,호반건설 광주시 계림 구역 정비 사업 수주 공사비 원 최고 개동 아파트 가구 규모 ...
3,0,성동조선해양 HSG중공업과 인수합병 MOU 체결 HSG중공업이 성동조선해양 인수를 ...,성동 조선 해양 중공업 인수 합병 체결 중공업 성동 조선 해양 인수 위한 양해 각서...
4,0,2019년 10월 프랜차이즈 정보공개서 79개 신규등록 서울·경기·인천 지역 서울시...,년 월 프랜차이즈 정보 공개 개 신규 등록 서울 경기 인천 지역 서울시 경기도 새롭...


In [80]:
data_doc_title_content_tag = make_doc2vec_data(data, 'title_content', t_document=True)
data_doc_title_content = make_doc2vec_data(data, 'title_content')
data_doc_tok_tag = make_doc2vec_data(data, 'mecab_tok', t_document=True)
data_doc_tok = make_doc2vec_data(data, 'mecab_tok')

In [81]:
#title_content_doc2vec_model = training_doc2vec(data_doc_title_content)

In [82]:
#tok_doc2vec_model = training_doc2vec(data_doc_tok)

In [83]:
#title_content_doc2vec_model.save('./title_content_doc2vec_model.doc2vec')
#tok_doc2vec_model.save('./data_doc_tok.doc2vec')


In [87]:
user_1 = make_user_embedding([20, 21, 24, 29, 45, 82], data_doc_title_content, title_content_doc2vec_model) # 카테고리 0

In [93]:
result = get_recommened_contents(user_1, data_doc_title_content, title_content_doc2vec_model)
pd.DataFrame(result.title_content)

,title_content
21,자막뉴스 11월 수출 1년 전보다 14.3%↓ 지난달 우리나라 수출액은 441억 달...
5639,KT 호텔 객실용품 배달하는 AI 호텔 로봇 KT가 국내 최초 AI 호텔인 노보텔 ...
13348,KT 호텔 객실용품 배달하는 AI 호텔 로봇 상용화 KT가 국내 최초 AI 호텔인 ...
13375,KT 전국 스키장에서도 5G 서비스 즐기세요 KT가 본격적인 겨울 스키 시즌을 맞아...
5441,수출 12개월째 내리막…‘부진의 늪’ 언제 탈출 앵커 지난달 수출액이 또다시 줄어들...
13347,KT 호텔 객실용품 배달하는 AI 호텔 로봇 KT가 국내 최초 AI 호텔인 노보텔 ...
3948,수출 12개월째 내리막…‘부진의 늪’ 언제 탈출 앵커 지난달 수출액이 또다시 줄어들...
4005,이마트 미리 준비하는 크리스마스 서울 연합뉴스 류효림 기자 2일 서울 용산구 한강로...
17170,SKTMS 게임으로 코딩 교육 서울 뉴스1 민경석 기자 SK텔레콤이 마이크로소프트와...
780,스테이크 원하는 두께로 썰어드려요 서울 뉴스1 구윤성 기자 1일 서울 이마트 용산점...


In [96]:
data.loc[[20, 21, 24, 29, 45, 82], :]

,category,title_content,mecab_tok
20,0,기업 78% 국회 경제입법 활동 낙제 수준 국내 기업들이 20대 국회가 거둔 경제입...,기업 국회 경제 입법 활동 낙제 수준 국내 기업 대 국회 거둔 경제 입법 사회 갈등...
21,0,자막뉴스 11월 수출 1년 전보다 14.3%↓ 지난달 우리나라 수출액은 441억 달...,자막 뉴스 월 수출 년 전 지난달 우리나라 수출액 달러 지난해 개월 내리막 해 수출...
24,0,맞벌이 부부는 한 명에 카드 사용 집중해야 유리 금융감독원은 신용카드 소득공제에서 ...,맞벌이 부부 명 카드 사용 집중 해야 유리 금융감독원 신용카드 소득 공제 맞벌이 부...
29,0,SK 중국 투자사와 1조원 펀드 조성...미래 먹거리 발굴 집중 SK그룹이 중국 최...,SK 중국 투자 사 원 펀드 조성 미래 먹거리 발굴 집중 그룹 중국 최대 글로벌 투...
45,0,수출 12개월째 내리막…‘부진의 늪’ 언제 탈출 앵커 지난달 수출액이 지난해 같은 ...,수출 개월 내리막 부진 늪 언제 탈출 앵커 지난달 수출액 지난해 같 기간 또 다시 ...
82,0,박영선 중기부 장관 프랑스 경제인 면담 서울 연합뉴스 박영선 중소벤처기업부 장관이 ...,박영선 중기부 장관 프랑스 경제 면담 서울 연합뉴스 박영선 중소 벤처 기업 장관 지...
